In [ ]:
from datetime import datetime
from tqdm import trange
from typing import List, Tuple, Dict
from pprint import pprint
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization, acquisition
import random

# Set seeds for reproducibility
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Ensure deterministic behavior in PyTorch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
NDVI_PATH = "../data/PROCESSED/ndvi.csv"
PROD_PATH = "../data/PROCESSED/manhuacu.csv"

In [ ]:
# MLP Hyperparameters
MLP_WINDOW_SIZE = 10
MLP_BATCH_SIZE = 8
MLP_BASE_HIDDEN_SIZE = 64
MLP_EPOCHS = 400
MLP_LEARNING_RATE = 1e-3
MLP_DROPOUT = 0.2

# Best MLP Hyperparameters
# MLP_WINDOW_SIZE = 10
# MLP_BATCH_SIZE = 4
# MLP_BASE_HIDDEN_SIZE = 32
# MLP_EPOCHS = 300
# MLP_LEARNING_RATE = 2e-3

# Validation Loss: 0.1911

# LSTM Hyperparameters
LSTM_WINDOW_SIZE = 10
LSTM_HIDDEN_SIZE = 32
LSTM_NUM_LAYERS = 2
LSTM_DROPOUT = 0.2
LSTM_EPOCHS = 1000
LSTM_BATCH_SIZE = 4
LSTM_LEARNING_RATE = 1e-4

# Best LSTM Hyperparameters So Far
# LSTM_WINDOW_SIZE = 20
# LSTM_HIDDEN_SIZE = 32
# LSTM_NUM_LAYERS = 2
# LSTM_DROPOUT = 0.2
# LSTM_EPOCHS = 1000
# LSTM_BATCH_SIZE = 16
# LSTM_LEARNING_RATE = 5e-5

# Validation Loss: 0.2122

# Bayesian Optimization Hyperparameters
INIT_POINTS = 3
N_ITER = 15


# Computation
LSTM_DROPOUT = LSTM_DROPOUT if LSTM_NUM_LAYERS > 1 else 0

In [ ]:
def get_day_of_year_index(date: datetime):
    """Convert date to day of year."""
    return datetime(date.year, date.month, date.day).timetuple().tm_yday - 1


def get_sin_cos(x: float):
    """Convert x to sin and cos."""
    rad = 2 * np.pi * x
    return (np.sin(rad), np.cos(rad))


def encode_date(date: datetime):
    is_leap_year = 1 if date.year % 4 == 0 else 0
    total_year_days = 366 if is_leap_year else 365
    day_index = get_day_of_year_index(date)
    return get_sin_cos(day_index / total_year_days)


# Test
print("Encoding date 2020-01-01")
print(encode_date(datetime(2020, 1, 1)))  # (0.0, 1.0)
print("\n")
print("Encoding date 2020-06-01")
print(encode_date(datetime(2020, 6, 1)))  # (0.5, 0.0)
print("\n")
print("Encoding date 2020-12-31")
print(encode_date(datetime(2020, 12, 31)))  # (0.9999999999999999, 1.0)
print("\n")

## 1. Carregar e Pré-processar Dados

### 1.1. Carregar e pre-processar os Dados

In [ ]:
NDVI = pd.read_csv(NDVI_PATH)
NDVI

In [ ]:
NDVI["N_Observations"] = NDVI.groupby("Year")["Data"].transform("count")

NDVI[["Date_sin", "Date_cos"]] = NDVI["Data"].apply(
    lambda x: pd.Series(encode_date(datetime.strptime(x, "%Y-%m-%d")))
)

# Assert order by Data (ascending)
NDVI = NDVI.sort_values(by="Data", ascending=True)

NDVI = NDVI[(NDVI["Year"] >= 2000) & (NDVI["Year"] <= 2023)]

NDVI

In [ ]:
PROD = pd.read_csv(PROD_PATH)
PROD = PROD[(PROD["Year"] >= 2000) & (PROD["Year"] <= 2023)]
# max_productivity = PROD["Productivity (kg/ha)"].max()
# PROD["Normalized_productivity"] = PROD["Productivity (kg/ha)"] / max_productivity
PROD

### 1.2. Visualizar os dados

In [ ]:
NDVI.plot(x="Data", y="NDVI", title="NDVI over time", figsize=(15, 5))

### 2.2. Preparar Datasets

#### 2.1.1. Normalização

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalizer dados NDVI
NDVI["Year_norm"] = NDVI["Year"].copy()

ndvi_scaler = StandardScaler().fit(NDVI[["NDVI", "Year"]].values)
NDVI[["NDVI_norm", "Year_norm"]] = ndvi_scaler.transform(NDVI[["NDVI", "Year"]].values)

NDVI

In [ ]:
# Normalizar produtividade
PROD["Year_norm"] = NDVI["Year"].copy()

prod_scaler = StandardScaler().fit(PROD[["Productivity (kg/ha)", "Year"]].values)
PROD[["Productivity_norm", "Year_norm"]] = prod_scaler.transform(
    PROD[["Productivity (kg/ha)", "Year"]].values
)
PROD

In [ ]:
class DatasetYearOfLast(torch.utils.data.Dataset):
    def __init__(self, ndvi_df, prod_df, window_size=LSTM_WINDOW_SIZE):
        self.ndvi_df = ndvi_df.copy().reset_index(drop=True)
        self.window_size = window_size
        self.prod_df = prod_df

        # Prepare windows grouped by year
        self.samples = []
        self.available_years = ndvi_df["Year"].unique().tolist()

        for idx, row in self.ndvi_df.iterrows():
            window = ndvi_df.iloc[idx : idx + window_size]

            if len(window) < window_size:
                break

            last_of_window = self.ndvi_df.iloc[idx + window_size - 1]

            if last_of_window["Year"] == row["Year"] or (
                last_of_window["Year"] == row["Year"] + 1
                and row["Year"] + 1 in self.available_years
            ):
                self.samples.append((window, last_of_window["Year"]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        window, year = self.samples[idx]
        ndvi = window[["NDVI", "Date_sin", "Date_cos", "Year_norm"]].values

        prod = self.prod_df[self.prod_df["Year"] == year]["Productivity_norm"].values[0]

        return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
            prod, dtype=torch.float32
        )

    def get_last_window_of_year(self, year) -> np.ndarray:
        """
        Retorna a última janela do ano
        """

        ndvi = self.ndvi_df[self.ndvi_df["Year"] == year]
        if ndvi.empty:
            raise ValueError(f"Year {year} not found in dataset")
        if len(ndvi) < self.window_size:
            raise ValueError(
                f"Year {year} has only {len(ndvi)} observations, less than window size {self.window_size}"
            )
        return (
            ndvi.iloc[-self.window_size :][
                ["NDVI", "Date_sin", "Date_cos", "Year_norm"]
            ].values,
            self.prod_df[self.prod_df["Year"] == year]["Productivity_norm"].values[0],
        )


years_validation = [2004, 2010, 2016, 2022]
years_test = [2005, 2011, 2017, 2023]
years_train = PROD[~PROD["Year"].isin(years_validation + years_test)]["Year"].unique()


# Datasets better
dataset_train = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_train)], PROD, MLP_WINDOW_SIZE
)
dataset_validation = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_validation)], PROD, MLP_WINDOW_SIZE
)
mlp_test_dataset = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_test)], PROD, MLP_WINDOW_SIZE
)

lstm_train_dataset_year_of_last = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_train)], PROD, LSTM_WINDOW_SIZE
)
lstm_validation_dataset_year_of_last = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_validation)], PROD, LSTM_WINDOW_SIZE
)
lstm_test_dataset = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_test)], PROD, LSTM_WINDOW_SIZE
)


print(dataset_validation[0])
print("\n")
print(dataset_validation.get_last_window_of_year(2004))

In [ ]:
# Sanity check
print("Last window of 2017 in the DataFrame:")
print(
    NDVI[NDVI["Year"] == 2017][["NDVI", "Date_sin", "Date_cos", "Year_norm"]].tail(
        LSTM_WINDOW_SIZE
    )
)

print(
    f"\nProdutivity 2017: {PROD[PROD['Year'] == 2017]['Productivity (kg/ha)'].values[0]}"
)
print(
    f"Produtivity 2017 (normalized): {PROD[PROD['Year'] == 2017]['Productivity_norm'].values[0]}"
)

print(f"\nLast window of 2017 in the DatasetYearOfLast (values should match exactely):")
print(lstm_test_dataset.get_last_window_of_year(2017))

assert (
    lstm_test_dataset.get_last_window_of_year(2017)[0]
    == NDVI[NDVI["Year"] == 2017][["NDVI", "Date_sin", "Date_cos", "Year_norm"]]
    .tail(LSTM_WINDOW_SIZE)
    .values
).all(), "\n❌ Sanity check failed! Please check the DatasetYearOfLast class"
print(
    "\n✅ Sanity check passed for LSTM! You can look values by yourself if you want to double check."
)

## 3. Model training

### 3.1. Multi-layer Perceptron

Essa rede é uma feedforward perceptron multi-layer comum (1 camada interna).

As entradas são os 20 últimos NDVIs do ano, a saída é a produtividade prevista (kg/ha).

In [ ]:
class AmplifiedTanh(nn.Module):
    def __init__(self, amplification_factor=1.0):
        super().__init__()
        self.amplification_factor = amplification_factor

    def forward(self, x):
        return self.amplification_factor * torch.tanh(x)


def load_mlp_datasets(window_size=MLP_WINDOW_SIZE):
    dataset_train = DatasetYearOfLast(
        NDVI[NDVI["Year"].isin(years_train)], PROD, window_size
    )
    dataset_validation = DatasetYearOfLast(
        NDVI[NDVI["Year"].isin(years_validation)], PROD, window_size
    )
    return (
        dataset_train,
        dataset_validation,
    )


def load_mlp_loaders(
    dataset_train: DatasetYearOfLast,
    dataset_validation: DatasetYearOfLast,
    batch_size=MLP_BATCH_SIZE,
):
    """Load MLP datasets and return DataLoader objects."""
    train_loader = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True
    )
    valid_loader = torch.utils.data.DataLoader(
        dataset_validation, batch_size=batch_size, shuffle=False, drop_last=True
    )
    return train_loader, valid_loader


def init_linear_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


def init_mlp_network(
    base_hidden_size=MLP_BASE_HIDDEN_SIZE,
    window_size=MLP_WINDOW_SIZE,
    dropout=MLP_DROPOUT,
    init_weights=True,
):
    """Initialize MLP network with given base hidden size and window size."""
    mlp_network = nn.Sequential(
        nn.Flatten(start_dim=1, end_dim=-1),
        nn.Linear(window_size * 4, base_hidden_size),
        nn.Dropout(dropout),
        nn.ReLU(),
        nn.Linear(base_hidden_size, base_hidden_size // 2),
        nn.Dropout(dropout),
        nn.ReLU(),
        nn.Linear(base_hidden_size // 2, 1),
        AmplifiedTanh(amplification_factor=1.5),
    )

    init_weights and mlp_network.apply(init_linear_weights)

    return mlp_network


# for name, param in mlp_network.named_parameters():
#     print(f"{name}: {param}")

# Step-by-step debug the MLP
# x = torch.randn(20, 4)
# print(f"Input shape: {x.shape}\n{x}\n")
# for i, layer in enumerate(mlp_network):
#     x = layer(x)
#     print(f"After layer {i} ({layer.__class__.__name__}): {x.shape}\n{x}\n")

In [ ]:
best_mlp_model = None
best_mlp_loss = float("inf")
best_mlp_losses_train = []
best_mlp_losses_validation = []


def train_mlp(
    epochs: int = MLP_EPOCHS,
    batch_size: int = MLP_BATCH_SIZE,
    learning_rate: float = MLP_LEARNING_RATE,
    window_size: int = MLP_WINDOW_SIZE,
    base_hidden_size: int = MLP_BASE_HIDDEN_SIZE,
    dropout: float = MLP_DROPOUT,
    verbose: bool = False,
):
    best_model_found = False
    # torch.manual_seed(1)
    global best_mlp_model
    global best_mlp_loss
    global best_mlp_losses_train
    global best_mlp_losses_validation
    train_ds, validation_ds = load_mlp_datasets(window_size=window_size)
    train_loader, validation_loader = load_mlp_loaders(
        train_ds, validation_ds, batch_size=batch_size
    )

    mlp_network = init_mlp_network(
        base_hidden_size=base_hidden_size, window_size=window_size, dropout=dropout
    ).to(device)
    optimizer = optim.Adam(mlp_network.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    mlp_losses_validation = []
    mlp_losses_train = []
    best_loss = float("inf")
    saved_epoch = 0

    range_fn = trange if verbose else range
    for i in range_fn(epochs):
        epoch_losses_train = []

        mlp_network.train()
        for ndvi, prod in train_loader:
            ndvi, prod = ndvi.to(device), prod.to(device)
            optimizer.zero_grad()
            pred = mlp_network(ndvi)
            loss = loss_fn(pred, prod.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_losses_train.append(loss.item())

        epoch_losses_validation = []
        mlp_network.eval()
        with torch.no_grad():
            for ndvi, prod in validation_loader:
                ndvi, prod = ndvi.to(device), prod.to(device)
                pred = mlp_network(ndvi)
                loss = loss_fn(pred, prod.unsqueeze(1))
                epoch_losses_validation.append(loss.item())

            if np.mean(epoch_losses_validation) < best_loss:
                best_loss = np.mean(epoch_losses_validation)
                saved_epoch = i + 1
                torch.save(mlp_network.state_dict(), "mlp.pth")

            if np.mean(epoch_losses_validation) < best_mlp_loss:
                best_model_found = True
                best_mlp_model = deepcopy(mlp_network)
                best_mlp_loss = np.mean(epoch_losses_validation)

        mlp_losses_train.append(np.mean(epoch_losses_train))
        mlp_losses_validation.append(np.mean(epoch_losses_validation))

    if best_model_found:
        best_mlp_losses_train = mlp_losses_train.copy()
        best_mlp_losses_validation = mlp_losses_validation.copy()

    verbose and print(
        f"\n\nSaved MLP model\tepoch: {saved_epoch}\tvalidation loss: {best_loss:.4f}"
    )

    mlp_network.load_state_dict(torch.load("mlp.pth"))

    return best_loss


def get_best_mlp_loss(**kwargs):
    """
    Get the best loss from training
    """
    return -train_mlp(
        # epochs=int(kwargs["epochs"]),
        batch_size=int(kwargs["batch_size"]),
        learning_rate=kwargs["learning_rate"],
        window_size=int(kwargs["window_size"]),
        base_hidden_size=int(kwargs["base_hidden_size"]),
        dropout=kwargs["dropout"],
        verbose=False,
    )


parameter_bounds = {
    # "epochs": (200, 600),
    "batch_size": (1, 12.999999999),
    "learning_rate": (1e-5, 1e-2),
    "window_size": (5, 20.999999999),
    "base_hidden_size": (16, 128.999999999),
    "dropout": (0.0, 0.3),
}

# Practical rule of thumb:
# kappa ≈ 1 → balanced between exploration and exploitation
# kappa < 1 → focus more on exploiting what you already know
# kappa > 2 → emphasize exploration heavily
acquisition_function = acquisition.UpperConfidenceBound(
    kappa=1.5,  # Start with slight exploration
    exploration_decay=0.76,  # Reduce kappa by 5% per step
    exploration_decay_delay=2,  # Only start decay after 2 steps
)

optimizer = BayesianOptimization(
    f=get_best_mlp_loss,
    pbounds=parameter_bounds,
    acquisition_function=acquisition_function,
    random_state=1,
)

optimizer.maximize(
    init_points=INIT_POINTS,
    n_iter=N_ITER,
)

In [ ]:
best_mlp_params = {**optimizer.max["params"]}
print(f"\n\nBest MLP parameters:")
pprint(best_mlp_params)

torch.save(best_mlp_model.state_dict(), "mlp.pth")
best_mlp_model = best_mlp_model.to(device)

In [ ]:
def plot_loss(train_losses, validation_losses):
    """
    Plots the training and validation losses over epochs.

    Args:
        train_losses (list): List of training loss values for each epoch.
        validation_losses (list): List of validation loss values for each epoch.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(validation_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Over Epochs")
    plt.legend()
    plt.grid(True)
    plt.show()


plot_loss(best_mlp_losses_train, best_mlp_losses_validation)

### 3.2. LSTM

In [ ]:
import random

random.seed(1)
def cat_name_generator(gender: str): return random.choice({"macho": ["Thor", "Loki", "Zeus", "Hades", "Apolo", "Ares", "Hermes", "Poseidon", "Hércules", "Aquiles", "Ulisses", "Atlas", "Perseu", "Orfeu", "Eros", "Hefesto", "Dionísio", "Héracles", "Cronos", "Prometeu", "Teseu", "Orfeu", "Eolo"], "femea": ["Afrodite", "Artemis", "Deméter", "Hera", "Perséfone", "Atena", "Héstia", "Eris", "Selene", "Gaia", "Tétis", "Eurídice", "Calipso", "Medusa", "Circe"]}[gender])

print("Nome de gato macho:", cat_name_generator("macho"))
print("Nome de gato fêmea:", cat_name_generator("femea"))

In [ ]:
import torch.nn as nn
import torch.optim as optim
import numpy as np


# Define model with Linear layer
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=LSTM_DROPOUT):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, 1)  # Output a single value

    def forward(self, x, hidden_n=None, hidden_c=None):
        if hidden_n is None or hidden_c is None:
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :])  # Get output of last time step
        else:
            out, (hidden_n, hidden_c) = self.lstm(x, (hidden_n, hidden_c))
            out = self.fc(out[:, -1, :])  # Get output of last time step
            return out, (hidden_n, hidden_c)


def get_lstm_datasets(window_size=LSTM_WINDOW_SIZE):
    lstm_train_dataset_year_of_last = DatasetYearOfLast(
        NDVI[NDVI["Year"].isin(years_train)], PROD, window_size
    )
    lstm_validation_dataset_year_of_last = DatasetYearOfLast(
        NDVI[NDVI["Year"].isin(years_validation)], PROD, window_size
    )
    return (
        lstm_train_dataset_year_of_last,
        lstm_validation_dataset_year_of_last,
    )


def get_lstm_dataloaders(
    train_ds: DatasetYearOfLast,
    validation_ds: DatasetYearOfLast,
    batch_size=LSTM_BATCH_SIZE,
):
    lstm_train_loader_year_of_last = torch.utils.data.DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
    )
    lstm_validation_loader_year_of_last = torch.utils.data.DataLoader(
        validation_ds,
        batch_size=4,
        shuffle=False,
        drop_last=True,
    )
    return lstm_train_loader_year_of_last, lstm_validation_loader_year_of_last


def init_lstm_weights(m):
    if isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)


def init_lstm(
    hidden_size=LSTM_HIDDEN_SIZE, num_layers=LSTM_NUM_LAYERS, dropout=LSTM_DROPOUT
):
    lstm_model = LSTMRegressor(
        input_size=4, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout
    ).to(device)

    lstm_model.apply(init_lstm_weights)
    lstm_model.apply(init_linear_weights)
    return lstm_model


best_lstm_model = None
best_lstm_loss = float("inf")
best_lstm_losses_train = []
best_lstm_losses_validation = []


def train_lstm(
    epochs=LSTM_EPOCHS,
    hidden_size=LSTM_HIDDEN_SIZE,
    num_layers=LSTM_NUM_LAYERS,
    learning_rate=LSTM_LEARNING_RATE,
    dropout=LSTM_DROPOUT,
    batch_size=LSTM_BATCH_SIZE,
    window_size=LSTM_WINDOW_SIZE,
    verbose=False,
):
    best_model_found = False
    global best_lstm_model
    global best_lstm_loss
    global best_lstm_losses_train
    global best_lstm_losses_validation

    train_ds, validation_ds = get_lstm_datasets(window_size=window_size)

    lstm_train_loader, lstm_validation_loader = get_lstm_dataloaders(
        train_ds=train_ds, validation_ds=validation_ds, batch_size=batch_size
    )

    lstm_model = init_lstm(
        num_layers=num_layers,
        hidden_size=hidden_size,
        dropout=0.0 if num_layers == 1 else dropout,
    )

    optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    lstm_losses_train = []
    lstm_losses_validation = []
    best_loss = float("inf")
    saved_epoch = 0

    it = trange if verbose else range
    for i in it(epochs):
        epoch_losses_train = []

        # h_n = torch.zeros(LSTM_NUM_LAYERS, LSTM_BATCH_SIZE, LSTM_HIDDEN_SIZE).to(
        #     device
        # )  # Hidden state
        # h_c = torch.zeros(LSTM_NUM_LAYERS, LSTM_BATCH_SIZE, LSTM_HIDDEN_SIZE).to(
        #     device
        # )  # Cell state

        lstm_model.train()
        for ndvi, prod in lstm_train_loader:
            ndvi, prod = ndvi.to(device), prod.to(device)
            optimizer.zero_grad()
            # pred, (h_n, h_c) = lstm_model(
            #     ndvi, h_n.detach(), h_c.detach()
            # )

            # Verificar se isto está certo
            # last_pred está correto?
            pred = lstm_model(ndvi)
            last_pred = pred[:, -1]
            loss = loss_fn(last_pred, prod)
            loss.backward()
            nn.utils.clip_grad_norm_(lstm_model.parameters(), 1.0)
            optimizer.step()
            epoch_losses_train.append(loss.item())

        epoch_losses_validation = []
        lstm_model.eval()
        with torch.no_grad():
            for ndvi, prod in lstm_validation_loader:
                ndvi, prod = ndvi.to(device), prod.to(device)
                pred = lstm_model(ndvi)
                last_pred = pred[:, -1]  # Get the last prediction
                loss = loss_fn(last_pred, prod)
                epoch_losses_validation.append(loss.item())

            if np.mean(epoch_losses_validation) < best_loss:
                best_loss = np.mean(epoch_losses_validation)
                saved_epoch = i + 1
                torch.save(lstm_model.state_dict(), "lstm.pth")

            if np.mean(epoch_losses_validation) < best_lstm_loss:
                best_model_found = True
                best_lstm_model = deepcopy(lstm_model)
                best_lstm_loss = np.mean(epoch_losses_validation)

        lstm_losses_train.append(np.mean(epoch_losses_train))
        lstm_losses_validation.append(np.mean(epoch_losses_validation))

    if best_model_found:
        best_lstm_losses_train = lstm_losses_train.copy()
        best_lstm_losses_validation = lstm_losses_validation.copy()

    verbose and print(
        f"\n\nSaved LSTM model\tepoch: {saved_epoch}\tvalidation loss: {best_loss:.4f}"
    )

    lstm_model.load_state_dict(torch.load("lstm.pth"))

    return best_loss


def get_best_loss(**kwargs):
    """
    Get the best loss from training
    """
    return -train_lstm(
        # epochs=int(kwargs["epochs"]),
        hidden_size=int(kwargs["hidden_size"]),
        num_layers=int(kwargs["num_layers"]),
        learning_rate=kwargs["learning_rate"],
        dropout=kwargs["dropout"],
        batch_size=int(kwargs["batch_size"]),
        window_size=int(kwargs["window_size"]),
    )


parameter_bounds = {
    "hidden_size": (16, 128.9999999999),  # 16 to 128
    "num_layers": (1, 2.9999999999),  # 1 or 2
    "learning_rate": (1e-6, 1e-3),
    "batch_size": (1, 16.9999999999),
    "dropout": (0.0, 0.3),
    # "epochs": (200, 1000.9999999999),  # (200, 1000.9999999999) 200 to 1000
    "window_size": (4, 20.9999999999),  # 4 to 20
}

acquisition_function = acquisition.UpperConfidenceBound(
    kappa=1.5,  # Start with slight exploration
    exploration_decay=0.76,  # Reduce kappa by 5% per step
    exploration_decay_delay=2,  # Only start decay after 2 steps
)

optimizer = BayesianOptimization(
    f=get_best_loss,
    pbounds=parameter_bounds,
    acquisition_function=acquisition_function,
    random_state=1,
)

optimizer.maximize(
    init_points=INIT_POINTS,
    n_iter=N_ITER,
)

In [ ]:
best_lstm_params = {**optimizer.max["params"]}
print(f"\n\nBest LSTM parameters:")
pprint(best_lstm_params)

best_lstm_model = best_lstm_model.to(device)

In [ ]:
plot_loss(best_lstm_losses_train, best_lstm_losses_validation)

## 4. Avaliação do Modelo

In [ ]:
mlp_test_dataset = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_test)], PROD, int(best_mlp_params["window_size"])
)
lstm_test_dataset = DatasetYearOfLast(
    NDVI[NDVI["Year"].isin(years_test)], PROD, int(best_lstm_params["window_size"])
)

mlp_test_loader = torch.utils.data.DataLoader(
    mlp_test_dataset, batch_size=4, shuffle=False, drop_last=True
)
lstm_test_loader = torch.utils.data.DataLoader(
    lstm_test_dataset, batch_size=4, shuffle=False, drop_last=True
)

# test MLP
test_losses_mlp = []
best_mlp_model.eval()
loss_fn = nn.MSELoss()
for ndvi, prod in mlp_test_loader:
    ndvi, prod = ndvi.to(device), prod.to(device)
    pred = best_mlp_model(ndvi)
    loss = loss_fn(pred, prod.unsqueeze(1))
    test_losses_mlp.append(loss.item())
print(f"MLP test loss: {np.mean(test_losses_mlp):.4f}")

# test LSTM
test_losses_lstm = []
best_lstm_model.eval()
for ndvi, prod in lstm_test_loader:
    ndvi, prod = ndvi.to(device), prod.to(device)
    pred = best_lstm_model(ndvi)
    last_pred = pred[:, -1]  # Get the last prediction
    loss = loss_fn(last_pred, prod)
    test_losses_lstm.append(loss.item())
    
print(f"LSTM test loss: {np.mean(test_losses_lstm):.4f}")

In [ ]:
best_mlp_model.eval()
all_preds = []
real_prods = []

with torch.no_grad():
    for ndvi, prod in mlp_test_loader:
        ndvi, prod = ndvi.to(device), prod.to(device)
        pred = best_mlp_model(ndvi)
        all_preds.append(pred.cpu().numpy())
        real_prods.append(prod.cpu().numpy())

# Flatten the predictions and real productions
all_preds = np.concatenate(all_preds, axis=0).flatten()
real_prods = np.concatenate(real_prods, axis=0).flatten()

# Print predictions and real productions
# print("LSTM Predictions for all test dataset:")
# print(all_preds)
# print("\nReal Production (normalized) for all test dataset:")
# print(real_prods)

plt.figure(figsize=(12, 6))
plt.plot(real_prods, label="Real Production (normalized)", marker="o")
plt.plot(all_preds, label="MLP Predictions (normalized)", marker="x")
plt.xlabel("Index")
plt.ylabel("Normalized Values")
plt.title("MLP Predictions vs Real Production")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
best_lstm_model.eval()
all_preds = []
real_prods = []

with torch.no_grad():
    for ndvi, prod in lstm_test_loader:
        ndvi, prod = ndvi.to(device), prod.to(device)
        pred = best_lstm_model(ndvi)
        all_preds.append(pred.cpu().numpy())
        real_prods.append(prod.cpu().numpy())

# Flatten the predictions and real productions
all_preds = np.concatenate(all_preds, axis=0)
real_prods = np.concatenate(real_prods, axis=0)

# Print predictions and real productions
# print("LSTM Predictions for all test dataset:")
# print(all_preds)
# print("\nReal Production (normalized) for all test dataset:")
# print(real_prods)

plt.figure(figsize=(12, 6))
plt.plot(real_prods, label="Real Production (normalized)", marker="o")
plt.plot(all_preds, label="LSTM Predictions (normalized)", marker="x")
plt.xlabel("Index")
plt.ylabel("Normalized Values")
plt.title("LSTM Predictions vs Real Production")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print("MLP test loss:", np.mean(test_losses_mlp))
for y_test in years_test:
    ndvi, prod = mlp_test_dataset.get_last_window_of_year(y_test)
    ndvi = torch.tensor(ndvi, dtype=torch.float32).unsqueeze(0).to(device)
    pred = best_mlp_model(ndvi)
    last_pred = pred[:, -1].cpu().detach().numpy()[0]
    test_losses_mlp.append(last_pred)
    print(f"\n[MLP] Predicted (normalized) productivity for {y_test}: {last_pred:.4f}")
    print(f"[MLP] Real productivity (normalized) for {y_test}: {prod:.4f}")

print("\n\nLSTM test loss:", np.mean(test_losses_lstm))
for y_test in years_test:
    ndvi, prod = lstm_test_dataset.get_last_window_of_year(y_test)
    ndvi = torch.tensor(ndvi, dtype=torch.float32).unsqueeze(0).to(device)
    pred = best_lstm_model(ndvi)
    last_pred = pred[:, -1].cpu().detach().numpy()[0]
    print(f"\n[LSTM] Predicted productivity (normalized) for {y_test}: {last_pred:.4f}")
    print(f"[LSTM] Real productivity (normalized) for {y_test}: {prod:.4f}")